<a href="https://colab.research.google.com/github/rachit-shah/Apparent-Age-Prediction-using-CNN/blob/master/ADBI_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!cp gdrive/'My Drive'/bbc-fulltext.zip .
!cp gdrive/'My Drive'/glove.6B.zip .

In [23]:
!unzip bbc-fulltext.zip

Archive:  bbc-fulltext.zip
   creating: bbc/
   creating: bbc/business/
  inflating: bbc/business/001.txt    
  inflating: bbc/business/002.txt    
  inflating: bbc/business/003.txt    
  inflating: bbc/business/004.txt    
  inflating: bbc/business/005.txt    
  inflating: bbc/business/006.txt    
  inflating: bbc/business/007.txt    
  inflating: bbc/business/008.txt    
  inflating: bbc/business/009.txt    
  inflating: bbc/business/010.txt    
  inflating: bbc/business/011.txt    
  inflating: bbc/business/012.txt    
  inflating: bbc/business/013.txt    
  inflating: bbc/business/014.txt    
  inflating: bbc/business/015.txt    
  inflating: bbc/business/016.txt    
  inflating: bbc/business/017.txt    
  inflating: bbc/business/018.txt    
  inflating: bbc/business/019.txt    
  inflating: bbc/business/020.txt    
  inflating: bbc/business/021.txt    
  inflating: bbc/business/022.txt    
  inflating: bbc/business/023.txt    
  inflating: bbc/business/024.txt    
  inflating: bbc

In [0]:
!rm bbc/README.TXT

In [34]:
!ls bbc/entertainment/

001.txt  044.txt  087.txt  130.txt  173.txt  216.txt  259.txt  302.txt	345.txt
002.txt  045.txt  088.txt  131.txt  174.txt  217.txt  260.txt  303.txt	346.txt
003.txt  046.txt  089.txt  132.txt  175.txt  218.txt  261.txt  304.txt	347.txt
004.txt  047.txt  090.txt  133.txt  176.txt  219.txt  262.txt  305.txt	348.txt
005.txt  048.txt  091.txt  134.txt  177.txt  220.txt  263.txt  306.txt	349.txt
006.txt  049.txt  092.txt  135.txt  178.txt  221.txt  264.txt  307.txt	350.txt
007.txt  050.txt  093.txt  136.txt  179.txt  222.txt  265.txt  308.txt	351.txt
008.txt  051.txt  094.txt  137.txt  180.txt  223.txt  266.txt  309.txt	352.txt
009.txt  052.txt  095.txt  138.txt  181.txt  224.txt  267.txt  310.txt	353.txt
010.txt  053.txt  096.txt  139.txt  182.txt  225.txt  268.txt  311.txt	354.txt
011.txt  054.txt  097.txt  140.txt  183.txt  226.txt  269.txt  312.txt	355.txt
012.txt  055.txt  098.txt  141.txt  184.txt  227.txt  270.txt  313.txt	356.txt
013.txt  056.txt  099.txt  142.txt  185.txt  228.txt

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
%matplotlib inline
import nltk
nltk.download('popular')

In [285]:
!unzip glove.6B.zip
!ls

Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       
2pageSessions.csv  glove.6B.100d.txt  glove.6B.zip		 readme.txt
bbc		   glove.6B.200d.txt  __MACOSX			 sample_data
bbc-fulltext.zip   glove.6B.300d.txt  NewsAggregatorDataset.zip
gdrive		   glove.6B.50d.txt   newsCorpora.csv


In [247]:
#Read Data
categories = ['business','entertainment','politics','sport','tech']
df = pd.DataFrame([],columns=['category','id','text'])
for cat in categories:
  for file in os.listdir("bbc/"+cat):
      if file.endswith(".txt"):
          filepath = os.path.join("bbc/"+cat, file)
          text = open(filepath,'r', errors='ignore').read()
          s = pd.Series([cat,int(filepath.split('/')[-1][:-4]),text],index=['category','id','text'])
          df = df.append(s,ignore_index=True)

df.head()
          
      


,category,id,text
0,business,182,Weak data buffets French economy\n\nA batch of...
1,business,449,Wal-Mart fights back at accusers\n\nTwo big US...
2,business,403,WorldCom bosses' $54m payout\n\nTen former dir...
3,business,223,Gaming firm to sell UK dog tracks\n\nSix UK gr...
4,business,481,Christmas sales worst since 1981\n\nUK retail ...


In [243]:
'''
#WordCloud

from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text 
from wordcloud import WordCloud, STOPWORDS
cat = df['category'].unique()

for i in range(0,len(cat)):
    words = ' '.join(df.loc[df['category']==cat[i], 'text'])

    wordcloud = WordCloud( 
                          stopwords=STOPWORDS,
                          background_color='white',
                          width=800,
                          height=400
                ).generate(words)
    print(cat[i])
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud)
    plt.axis('off')
    plt.show()

words = ' '.join(df.loc[:, 'text'])

wordcloud = WordCloud( 
                      stopwords=STOPWORDS,
                      background_color='white',
                      width=800,
                      height=400
            ).generate(words)
print("ALL Categories:")
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()
'''

'\n#WordCloud\n\nfrom wordcloud import WordCloud\nimport matplotlib.pyplot as plt\nfrom sklearn.feature_extraction import text \nfrom wordcloud import WordCloud, STOPWORDS\ncat = df[\'category\'].unique()\n\nfor i in range(0,len(cat)):\n    words = \' \'.join(df.loc[df[\'category\']==cat[i], \'text\'])\n\n    wordcloud = WordCloud( \n                          stopwords=STOPWORDS,\n                          background_color=\'white\',\n                          width=800,\n                          height=400\n                ).generate(words)\n    print(cat[i])\n    plt.figure(figsize=(10, 5))\n    plt.imshow(wordcloud)\n    plt.axis(\'off\')\n    plt.show()\n\nwords = \' \'.join(df.loc[:, \'text\'])\n\nwordcloud = WordCloud( \n                      stopwords=STOPWORDS,\n                      background_color=\'white\',\n                      width=800,\n                      height=400\n            ).generate(words)\nprint("ALL Categories:")\nplt.figure(figsize=(10, 5))\nplt.imshow(wo

## Label Encoder
#### Use inverse_transform at the end after predicting

In [336]:
from sklearn.preprocessing import LabelEncoder
import pickle
import numpy as np
from sklearn.model_selection import train_test_split
y = df['category']
X = df.drop(['category','id'],axis=1)
X_train, X_test, y_train, y_test  = train_test_split(X,y,stratify=y, test_size=0.2, random_state=123)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

print('before: %s ...' %y_train[:5])

le = LabelEncoder()
le.fit(y_train)
y_train = le.transform(y_train)

print('after: %s ...' %y_train)

before: 1508       sport
1908        tech
2104        tech
2019        tech
1245    politics
Name: category, dtype: object ...
after: [3 4 4 ... 1 0 2] ...


In [337]:
print(np.bincount(y_train))
print(y.value_counts())

print(le.inverse_transform([i for i in range(5)]))

[408 309 333 409 321]
sport            511
business         510
politics         417
tech             401
entertainment    386
Name: category, dtype: int64
['business' 'entertainment' 'politics' 'sport' 'tech']


In [0]:
def preprocess(X):
  return X.map(lambda x: x.lower().split("\n")).map(lambda x: [y.split(". ") for y in x]).map(lambda x: [i.replace('\'','') for sl in x for i in sl if i is not ''])

In [339]:
X_train['text'] = preprocess(X_train['text'])
X_train['text']

0       [tigers wary of farrell gamble, leicester say ...
1       [supercomputer breaks speed record, the us is ...
2       [anti-tremor mouse stops pc shakes, a special ...
3       [who do you think you are?, the real danger is...
4       [blair defends terror law changes, the prime m...
5       [csa chief who quit still in job, the head of ...
6       [us bank boss hails genius smith, us federal r...
7       [argentina closes $102.6bn debt swap, argentin...
8       [the gaming world in 2005, if you have finishe...
9       [visa decision every 11 minutes, visa processi...
10      [scots suffer another injury blow, scotlands b...
11      [blairs hope for blunkett return, the events l...
12      [manics in charge of bbc 6 music, the manic st...
13      [wenger steps up row, arsene wenger has steppe...
14      [hospital suspends no welsh plan, an english h...
15      [baa support ahead of court battle, uk airport...
16      [african double in edinburgh, world 5000m cham...
17      [heart

In [0]:
#Stop Words and Lemmatization
import nltk.corpus
import string
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.stem import WordNetLemmatizer   
lemmatizer = WordNetLemmatizer()
stopw = nltk.corpus.stopwords.words('english')
punct = string.punctuation
punct = word_tokenize(punct)
punct += ['.','``','...','\'s','--','-','n\'t','\'']
stopw += punct
def token_stop(text):
    global stopw
    global lemmatizer
    words = word_tokenize(text)
    filtered = [lemmatizer.lemmatize(w) for w in words if not w in stopw]
    return filtered
  
X_train['text'] = X_train['text'].map(lambda x: [token_stop(i) for i in x]).map(lambda x: [i for sl in x for i in sl])
  

In [341]:
texts = np.array(X_train['text'])
MAX_SEQUENCE_LENGTH = 1000
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2
embeddings_index = {}
with open('glove.6B.100d.txt') as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, 'f', sep=' ')
        embeddings_index[word] = coefs

print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [342]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding, Flatten
from keras.models import Model
from keras.initializers import Constant

tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

labels = to_categorical(np.asarray(y_train))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

# split the data into a training set and a validation set
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]
nb_validation_samples = int(VALIDATION_SPLIT * data.shape[0])

xtrain = data[:-nb_validation_samples]
ytrain = labels[:-nb_validation_samples]
xval = data[-nb_validation_samples:]
yval = labels[-nb_validation_samples:]

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 28405 unique tokens.
Shape of data tensor: (1780, 1000)
Shape of label tensor: (1780, 5)


In [0]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [0]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [345]:
labels_index = {}
for i in range(5):
  name = le.inverse_transform([i])[0]
  labels_index[name] = i
labels_index

{'business': 0, 'entertainment': 1, 'politics': 2, 'sport': 3, 'tech': 4}

In [346]:

sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedded_sequences = embedding_layer(sequence_input)
x = Conv1D(128, 5, activation='relu')(embedded_sequences)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(5)(x)
x = Conv1D(128, 5, activation='relu')(x)
x = MaxPooling1D(35)(x)  # global max pooling
x = Flatten()(x)
x = Dense(128, activation='relu')(x)
preds = Dense(len(labels_index), activation='softmax')(x)

model = Model(sequence_input, preds)
model.compile(loss='categorical_crossentropy',
              optimizer='rmsprop',
              metrics=['acc'])

# happy learning!
model.fit(xtrain, ytrain, validation_data=(xval, yval),
          epochs=10, batch_size=128)

Train on 1424 samples, validate on 356 samples
Epoch 1/10
1424/1424 [==============================] - 16s 11ms/step - loss: 1.5685 - acc: 0.4157 - val_loss: 1.0281 - val_acc: 0.5646
Epoch 2/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.6512 - acc: 0.7549 - val_loss: 0.5209 - val_acc: 0.8287
Epoch 3/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.3451 - acc: 0.8792 - val_loss: 0.1592 - val_acc: 0.9635
Epoch 4/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.2251 - acc: 0.9171 - val_loss: 0.1292 - val_acc: 0.9719
Epoch 5/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.2498 - acc: 0.9192 - val_loss: 0.1375 - val_acc: 0.9691
Epoch 6/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.0790 - acc: 0.9726 - val_loss: 0.9254 - val_acc: 0.7331
Epoch 7/10
1424/1424 [==============================] - 15s 11ms/step - loss: 0.1661 - acc: 0.9417 - val_loss: 0.0825 - val_acc: 0.9747
E

In [348]:
X_test['text'] = preprocess(X_test['text'])
X_test['text'] = X_test['text'].map(lambda x: [token_stop(i) for i in x]).map(lambda x: [i for sl in x for i in sl])
test_text = np.array(X_test['text'])
tokenizer = Tokenizer(nb_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(test_text)
sequences = tokenizer.texts_to_sequences(test_text)

word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)
prediction = model.predict(data)
prediction

/usr/local/lib/python3.6/dist-packages/keras_preprocessing/text.py:178: UserWarning: The `nb_words` argument in `Tokenizer` has been renamed `num_words`.
  warnings.warn('The `nb_words` argument in `Tokenizer` '


Found 14490 unique tokens.
Shape of data tensor: (445, 1000)
Shape of label tensor: (1780, 5)


array([[0.43026203, 0.07141124, 0.02702082, 0.00890647, 0.46239948],
       [0.80635977, 0.02241753, 0.05791005, 0.0098392 , 0.10347339],
       [0.7553198 , 0.06388112, 0.16078942, 0.00230318, 0.01770654],
       ...,
       [0.19572994, 0.47767732, 0.06225896, 0.00856006, 0.25577372],
       [0.82450974, 0.01964773, 0.06259804, 0.04207886, 0.05116564],
       [0.9717814 , 0.00268447, 0.01266349, 0.00190804, 0.01096253]],
      dtype=float32)

In [356]:
count = 0
t = 0
for pred in prediction:
  p = pred.argmax()
  name = le.inverse_transform([p])
  if name == y_test.loc[count]:
    t+=1
  count+=1
    
print(t/count)

0.22696629213483147


In [0]:
preds = le.inverse_transform(prediction)


In [0]:
y_test = y_test.reset_index(drop=True)